# <font color="#418FDE" size="6.5" uppercase>**Lists And Tuples**</font>

>Last update: 20260101.
    
By the end of this Lecture, you will be able to:
- Describe how Python lists are implemented and how that affects operation complexity. 
- Analyze the time complexity of common list operations such as append, insert, and slice. 
- Differentiate between lists and tuples in terms of mutability, usage patterns, and performance. 


## **1. Python List Internals**

### **1.1. Dynamic Array Basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_01_01.jpg?v=1767329202" width="250">



>* Lists are dynamic arrays of object references
>* Contiguous layout makes index access consistently fast

>* Lists use pre-allocated fixed-size memory blocks
>* Appends fill spare slots until resizing is needed

>* List occasionally resizes to a larger block
>* Most appends are cheap, resizing causes rare spikes



### **1.2. Amortized List Appends**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_01_02.jpg?v=1767329211" width="250">



>* Most appends are cheap writes into spare space
>* Occasional costly resizes average out over time

>* Python over-allocates list space for future appends
>* Most appends are cheap; occasional resizes are amortized

>* Amortized appends make average cost constant-time
>* Lists stay fast except in rare tight loops



In [ ]:
#@title Python Code - Amortized List Appends

# Demonstrate amortized list appends with simple timing experiment.
# Show that many appends stay fast on average overall.
# Compare total time for building lists of different sizes.

# pip install numpy matplotlib seaborn pandas scikit-learn torch tensorflow.

# Import time module for measuring elapsed time precisely.
import time

# Define a helper function that appends many items to a list.
def build_list_with_appends(count):
    # Start timing before the append loop begins.
    start_time = time.perf_counter()
    # Create an empty list that will grow using appends.
    data_list = []
    # Append simple integers repeatedly to simulate growing data.
    for number in range(count):
        data_list.append(number)
    
    # Stop timing after all appends have completed successfully.
    end_time = time.perf_counter()
    # Return elapsed seconds for this entire append sequence.
    return end_time - start_time

# Choose different target sizes to compare total append behavior.
sizes = [1_000, 10_000, 100_000, 1_000_000]

# Print a header explaining what the following numbers represent.
print("Total time for building lists with repeated appends:")

# Loop through sizes and measure time for each list construction.
for count in sizes:
    # Measure elapsed time for building a list of given size.
    elapsed = build_list_with_appends(count)
    # Compute average time per append operation in microseconds.
    average_microseconds = (elapsed / count) * 1_000_000
    
    # Print size, total time, and average time per append.
    print(f"Size {count:>9}, total {elapsed:.4f} s, average {average_microseconds:.4f} µs")




### **1.3. Resizing and memory overhead**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_01_03.jpg?v=1767329225" width="250">



>* Lists over-allocate space to allow cheap appends
>* Occasional resizing copies elements and wastes some memory

>* Resizing sometimes causes slow, expensive copy operations
>* Most appends stay fast, with rare costly bursts

>* Extra reserved capacity trades memory for speed
>* Overhead matters more with tight memory or gigant lists



In [ ]:
#@title Python Code - Resizing and memory overhead

# Demonstrate list resizing behavior and memory overhead using simple appends.
# Show occasional expensive operations when underlying storage needs more space.
# Help visualize extra capacity compared with actual stored elements.

# pip install psutil for memory inspection if not already installed.

# Import required modules for measuring memory usage and timing operations.
import sys
import time

# Define a helper function that appends many items and measures timings.
def measure_appends(total_items, label_description):
    start_time = time.perf_counter()
    data_list = []

    # Append integers one by one and track occasional timing checkpoints.
    checkpoints = [1, total_items // 4, total_items // 2, total_items]
    times_recorded = {}

    for index_value in range(1, total_items + 1):
        data_list.append(index_value)
        if index_value in checkpoints:
            times_recorded[index_value] = time.perf_counter() - start_time

    # Measure approximate memory usage of the list container only.
    list_memory_bytes = sys.getsizeof(data_list)

    # Print summary showing size, memory, and timing checkpoints.
    print(f"\nSummary for {label_description}.")
    print(f"Items stored count: {len(data_list)} elements.")
    print(f"List container memory: {list_memory_bytes} bytes.")

    # Print timing information to hint at occasional slower growth operations.
    for checkpoint, elapsed in times_recorded.items():
        print(f"Time after {checkpoint} appends: {elapsed:.6f} seconds.")

# Run the measurement with a moderate number of appends for clarity.
measure_appends(20000, "list resizing demonstration")



## **2. List Operation Costs**

### **2.1. Indexing and Iteration**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_02_01.jpg?v=1767329254" width="250">



>* List indexing uses direct memory access, very fast
>* Index lookups stay constant time, regardless of size

>* Iterating touches every element, so time grows
>* Whole-list tasks like search or sum are linear

>* Index lookups stay fast, even on huge lists
>* Full scans grow with size; design algorithms carefully



In [ ]:
#@title Python Code - Indexing and Iteration

# Demonstrate list indexing constant time behavior with simple timing example.
# Demonstrate list iteration linear time behavior with simple timing example.
# Compare how total work grows when list size doubles repeatedly.

# pip install numpy matplotlib seaborn  # Not required for this simple script.

# Import time module for measuring simple elapsed durations.
import time

# Create base list with many elements for timing tests.
base_list = list(range(1_000_000))

# Define different sizes to test indexing and iteration costs.
sizes = [10_000, 20_000, 40_000, 80_000]

# Print header describing what the table columns represent.
print("Size, index_time_microseconds, iterate_time_milliseconds")

# Loop over each chosen size and measure operation durations.
for n in sizes:

    # Take prefix slice so each test uses first n elements.
    current = base_list[:n]

    # Choose a middle index to access repeatedly for fairness.
    middle_index = n // 2

    # Time many index operations to get measurable duration.
    start_index = time.perf_counter()

    # Perform repeated indexing using same middle index position.
    for _ in range(100_000):
        value = current[middle_index]

    # Compute elapsed time for indexing operations in microseconds.
    index_elapsed = (time.perf_counter() - start_index) * 1_000_000

    # Time a single full iteration over the current list slice.
    start_iter = time.perf_counter()

    # Sum elements to ensure Python actually visits every element.
    total = 0
    for item in current:
        total += item

    # Compute elapsed time for iteration in milliseconds units.
    iter_elapsed = (time.perf_counter() - start_iter) * 1_000

    # Print rounded timings showing growth patterns for both operations.
    print(f"{n}, {index_elapsed:.1f}, {iter_elapsed:.3f}")



### **2.2. Mid List Updates**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_02_02.jpg?v=1767329268" width="250">



>* Mid-list inserts and deletes require shifting elements
>* More elements shifted means higher, growing time cost

>* Inserting or deleting mid-list shifts many elements
>* More shifted elements means time grows linearly

>* Frequent mid-list updates can severely slow programs
>* Prefer end updates or alternative data structures



In [ ]:
#@title Python Code - Mid List Updates

# Demonstrate mid list insert and delete time costs clearly.
# Compare inserting at front versus appending at end operations.
# Show how operation time grows with list size.

# pip install numpy matplotlib seaborn optional packages here.

# Import required standard library modules only here.
import time

# Define a helper function measuring insert time here.
def measure_insert_front_time(size, repeats):
    total_duration = 0.0
    for _ in range(repeats):
        data = list(range(size))
        start = time.perf_counter()
        data.insert(0, -1)
        end = time.perf_counter()
        total_duration += end - start
    return total_duration / repeats

# Define a helper function measuring append time here.
def measure_append_end_time(size, repeats):
    total_duration = 0.0
    for _ in range(repeats):
        data = list(range(size))
        start = time.perf_counter()
        data.append(-1)
        end = time.perf_counter()
        total_duration += end - start
    return total_duration / repeats

# Choose list sizes showing growing mid update costs.
list_sizes = [1_000, 10_000, 100_000]

# Choose repeat count for smoother timing results.
repeats = 20

# Print table header describing measurements.
print("Size    insert_front_seconds    append_end_seconds")

# Loop through sizes and display average timings.
for size in list_sizes:
    insert_time = measure_insert_front_time(size, repeats)
    append_time = measure_append_end_time(size, repeats)
    print(f"{size:<8}{insert_time:<24.8f}{append_time:.8f}")



### **2.3. Slice and Copy Costs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_02_03.jpg?v=1767329282" width="250">



>* Slicing creates a new list, copying element references
>* Time cost depends on slice length, not list

>* Full list copies are whole-list slices
>* Copy time grows linearly and can bottleneck

>* Slices and copies use extra memory and time
>* Prefer indices or iterators to avoid duplication



In [ ]:
#@title Python Code - Slice and Copy Costs

# Demonstrate list slicing and copying time costs with simple timing examples.
# Show that slice cost depends on slice length, not original list size.
# Compare partial slice cost and full copy cost using different list sizes.
# pip install commands are unnecessary because this script uses only standard libraries.

# Import time module for measuring simple elapsed durations.
import time

# Create a large list representing many temperature readings in Fahrenheit degrees.
large_list = list(range(1_000_000))

# Create a smaller list representing fewer temperature readings in Fahrenheit degrees.
small_list = list(range(20_000))

# Define a helper function that times a slicing operation once.
def time_slice(source_list, start_index, end_index):
    start_time = time.perf_counter()
    slice_result = source_list[start_index:end_index]
    end_time = time.perf_counter()
    return end_time - start_time

# Time slicing ten elements from the large list near the beginning.
large_ten_time = time_slice(large_list, 100, 110)

# Time slicing ten elements from the small list near the beginning.
small_ten_time = time_slice(small_list, 100, 110)

# Time copying the entire large list using a full slice expression.
large_full_copy_time = time_slice(large_list, 0, len(large_list))

# Time copying the entire small list using a full slice expression.
small_full_copy_time = time_slice(small_list, 0, len(small_list))

# Print timing results showing similar costs for equal slice lengths.
print("Slice ten elements large list seconds:", round(large_ten_time, 7))

# Print timing results showing similar costs for equal slice lengths.
print("Slice ten elements small list seconds:", round(small_ten_time, 7))

# Print timing results showing larger cost for copying the entire large list.
print("Full copy large list seconds:", round(large_full_copy_time, 7))

# Print timing results showing smaller cost for copying the entire small list.
print("Full copy small list seconds:", round(small_full_copy_time, 7))



## **3. Practical Tuples**

### **3.1. Immutable Tuple Guarantees**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_03_01.jpg?v=1767329299" width="250">



>* Tuple contents never change after creation
>* Immutability prevents hidden bugs in large systems

>* Tuples keep related values fixed and trustworthy
>* Prevent silent changes, aiding debugging and history tracking

>* Immutable tuples can be safely shared across modules
>* Sharing tuples reduces copying, improves performance and clarity



In [ ]:
#@title Python Code - Immutable Tuple Guarantees

# Demonstrate tuple immutability guarantees with simple geographic and order examples.
# Compare tuple behavior with list mutability and accidental in place modifications.
# Show how shared immutable tuples avoid unexpected changes across different program parts.
# pip install some_required_library_if_needed_but_standard_library_is_sufficient_here.

# Create a geographic coordinate using an immutable tuple representation.
coordinate_tuple = (40.7128, -74.0060, "New York")

# Create the same coordinate using a mutable list representation.
coordinate_list = [40.7128, -74.0060, "New York"]

# Print both structures to compare their initial identical contents.
print("Original tuple coordinate:", coordinate_tuple)
print("Original list coordinate:", coordinate_list)

# Attempt to modify the tuple element and catch the raised exception.
try:
    coordinate_tuple[0] = 41.0000
except TypeError as error:
    print("Tuple modification failed, error type:", type(error).__name__)

# Modify the list element successfully, demonstrating mutability and changed state.
coordinate_list[0] = 41.0000
print("Modified list coordinate:", coordinate_list)

# Store an immutable order record tuple shared across different program components.
order_record = ("SKU123", 19.99, "2024-01-01 10:00")

# Simulate two modules reading the same shared immutable order record.
module_a_view = order_record
module_b_view = order_record

# Print both module views to show identical stable shared data.
print("Module A order view:", module_a_view)
print("Module B order view:", module_b_view)

# Show that both views remain unchanged because the underlying tuple is immutable.
print("Underlying order tuple unchanged:", order_record)



### **3.2. Tuple Dictionary Keys**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_03_02.jpg?v=1767329315" width="250">



>* Tuples are immutable, so they’re valid dictionary keys
>* Great for composite keys needing stable, hashable values

>* Tuple keys model multidimensional relationships for lookup
>* They act as compact, immutable composite identifiers

>* Immutable tuple keys prevent accidental dictionary corruption
>* Tuples give fast lookups and stable composite identifiers



In [ ]:
#@title Python Code - Tuple Dictionary Keys

# Demonstrate using tuples as dictionary keys safely and effectively.
# Compare tuple keys with list keys for dictionary usage clarity.
# Show composite keys representing city and year climate measurements.
# !pip install some_required_library_here.

# Create a dictionary using tuple keys for city and year.
climate_data = {("Boston", 2020): 55.3, ("Boston", 2021): 54.8}

# Access a value using the same tuple key components.
boston_2020_temp = climate_data[("Boston", 2020)]

# Print the retrieved temperature for the specific tuple key.
print("Boston 2020 average temperature Fahrenheit:", boston_2020_temp)

# Show that tuples with same contents are equal and hashable.
key_one = ("Denver", 2019)
key_two = ("Denver", 2019)

# Use both keys to access or set dictionary entries consistently.
climate_data[key_one] = 49.1

# Print value using the second equal tuple key.
print("Denver 2019 average temperature Fahrenheit:", climate_data[key_two])

# Demonstrate that lists cannot be used directly as dictionary keys.
try:
    invalid_dict = { ["Miami", 2022]: 77.5 }
except TypeError as error:

    # Print the error message showing unhashable list key problem.
    print("List key error type and message:", type(error).__name__, str(error))

# Finally, print all dictionary items showing tuple composite keys clearly.
print("All climate entries with tuple keys:", list(climate_data.items()))



### **3.3. Tuple Performance Tradeoffs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master Python Algorithms/Module_03/Lecture_A/image_03_03.jpg?v=1767329336" width="250">



>* Tuples are immutable and slightly more memory efficient
>* Best for static data; frequent changes favor lists

>* Indexing and looping cost are basically identical
>* Lists grow cheaply; tuples recreate and copy data

>* Use tuples for stable, read-heavy, fixed records
>* Use lists for changing, write-heavy, growing data



In [ ]:
#@title Python Code - Tuple Performance Tradeoffs

# Demonstrate tuple and list performance tradeoffs with simple timing comparisons.
# Show cost of repeatedly extending tuples versus efficiently appending to lists.
# Help choose correct sequence type for dynamic or static data workloads.

# pip install commands are unnecessary because this script uses only standard libraries.

# Import time module for measuring simple elapsed durations.
import time

# Define helper function for timing repeated list appends efficiently.
def time_list_appends(repetitions, operations_per_repetition):
    start_time = time.perf_counter()
    for _ in range(repetitions):
        data_list = []
        for value in range(operations_per_repetition):
            data_list.append(value)
    end_time = time.perf_counter()
    return end_time - start_time

# Define helper function for timing repeated tuple extensions inefficiently.
def time_tuple_extensions(repetitions, operations_per_repetition):
    start_time = time.perf_counter()
    for _ in range(repetitions):
        data_tuple = ()
        for value in range(operations_per_repetition):
            data_tuple = data_tuple + (value,)
    end_time = time.perf_counter()
    return end_time - start_time

# Set modest sizes to keep runtime quick and output readable.
repetitions = 50
operations_per_repetition = 200

# Measure total time for building many growing lists efficiently.
list_time_seconds = time_list_appends(repetitions, operations_per_repetition)

# Measure total time for building many growing tuples inefficiently.
tuple_time_seconds = time_tuple_extensions(repetitions, operations_per_repetition)

# Print clear comparison showing relative performance difference for dynamic workloads.
print("List append total seconds:", round(list_time_seconds, 6))

# Print tuple timing which should be noticeably slower for repeated growth.
print("Tuple extend total seconds:", round(tuple_time_seconds, 6))

# Print ratio showing how many times slower tuple growth was in this experiment.
print("Tuple growth relative slowdown:", round(tuple_time_seconds / list_time_seconds, 2), "times slower")



# <font color="#418FDE" size="6.5" uppercase>**Lists And Tuples**</font>


In this lecture, you learned to:
- Describe how Python lists are implemented and how that affects operation complexity. 
- Analyze the time complexity of common list operations such as append, insert, and slice. 
- Differentiate between lists and tuples in terms of mutability, usage patterns, and performance. 

In the next Lecture (Lecture B), we will go over 'Dicts And Sets'